# 2. Metabolic pathway visualizations with Escher

## Escher Python tutorial

The Escher Python package makes it easy to run and customize Escher visualizations from within a Jupyter Notebook or Jupyter lab. The Python package also works directly with COBRApy models. This tutorial introduces the main features. It is also a runnable notebook, which you can download here:

https://github.com/zakandrewking/escher/blob/master/docs/escher-python.ipynb

Once Escher and the appropriate extension are installed (next section), you should be able to run this notebook in Jupyter Notebook or Jupyter Lab.

If you don't use Jupyter, you can also save standalone visualizations as HTML files using Escher Python. An example is included at the end of the tutorial.

### Installation

You can install Escher Python with pip:

In [ ]:
pip install pytest-astropy

In [ ]:
pip install escher

When you pip install escher, the Jupyter notebook extension should be installed automatically. If that doesn't work, try:

In [ ]:
# The notebook extenstion should install automatically. You can check by running:
jupyter nbextension list
# Make sure you have version >=5 of the `notebook` package
pip install "notebook>=5"
# To manually install the extension
jupyter nbextension install --py escher
jupyter nbextension enable --py escher
# depending on you environment, you might need the `--sysprefix` flag with those commands

If you have issues, you can leave a bug report on GitHub or ask for help on Gitter.

### Load an Escher map
### To get started, we'll import Escher and COBRApy

In [ ]:
import escher
from escher import Builder
import cobra
from time import sleep

### Global configuration

Escher currently offers on global configuration option. We will consider adding more in the future. This option turns off the warning message if you leave or refresh this page. The default setting is False to help you avoid losing work.

In [ ]:
escher.rc['never_ask_before_quit'] = True

For most of this tutorial, all you need is the Builder class from escher.

### The Builder

In [ ]:
builder = Builder()

Builder is a Jupyter widget, so executing it in a cell will run the widget and embed an Escher map. You should see an empty Escher map in the next cell

In [ ]:
builder

### Finding models and maps

But the Builder isn't very useful without some data!

In general, Escher is meant for building your own maps. Maps are generated with the content of genome-scale models, and you can find many models to explore on the BiGG Models database: http://bigg.ucsd.edu

Let's download the red blood cell model iAB_RBC_283.

In [ ]:
model = 'iAB_RBC_283.json'

We can add this model to the Escher Builder with the model_json option.

In [ ]:
builder = Builder(
    model_json=model
)

Now, in the following map, click "Edit > Add reaction mode" and click anywhere to start drawing reactions.

In [ ]:
builder

Later, you can save this map by clicking "Map > Save map JSON", and you can load JSON files in a Builder with the map_json option.

### TIP: You can jump into full-screen mode by clicking the button on the map¶
In full-screen mode, keyboard shortcuts, mouse scrolling, and trackpad interactions are all activated.

To leave full-screen mode, press the escape key or click again.

## Maps and models from the Escher website¶

Testing and getting started, Escher also provides a few default maps and models. You can see which ones are available with the following functions. To use a map or model, pass the name to the Builder options map_name and model_name.

In [ ]:
escher.list_available_maps()

In [ ]:
escher.list_available_models()

We can load these maps and models by specifying map_name and model_name.

In [ ]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)

In [ ]:
builder

### Reactive options

The Escher Builder is a "reactive" widget, so it will update automatically when you change data and options. Let's create another empty builder.

In [ ]:
builder=Builder()

In [ ]:
builder

And this time, try running some of the following cells and look at the response in the map above.

In [ ]:
# Load an Escher map
builder.map_name = 'iJO1366.Central metabolism'

In [ ]:
# Load a COBRA model
builder.model_name = 'e_coli_core'

In [ ]:
# Find any reactions that are in the map and not in the model, and turn them red
builder.highlight_missing = True

COBRA models are available directly from BiGG Models. Download a model from BiGG in the JSON format and read it with COBRApy.

In [ ]:
builder.model = cobra.io.load_json_model('iJO1366.json')

In [ ]:
# Run FBA with the model and add the flux data to the map
solution = builder.model.optimize()
builder.reaction_data = solution.fluxes

In [ ]:
# Add some data for metabolites
builder.metabolite_data = solution.shadow_prices

In [ ]:
# Simplify the map by hiding some labels
builder.hide_secondary_metabolites = True
builder.hide_all_labels = True

In [ ]:
builder.reaction_scale = [
    { 'type': 'min', 'color': '#000000', 'size': 12 },
    { 'type': 'median', 'color': '#ffffff', 'size': 20 },
    { 'type': 'max', 'color': '#ff0000', 'size': 25 }
]

In [ ]:
builder.reaction_scale_preset = 'GaBuRd'

In [ ]:
# Make all the arrows three times as thick
builder.reaction_scale = [
    {k: v * 3 if k == 'size' else v for k, v in x.items()} 
    for x in builder.reaction_scale
]

In [ ]:
# some other things to try:
# builder.scroll_behavior = 'zoom'
# builder.reaction_styles = ['size']

### All options

The full list of options is provided in the Python API documentation:

https://escher.readthedocs.io/en/latest/python_api.html

For more-detailed descriptions of the Keyword Arguments, you can also refer to the JavaScript API docs:

https://escher.readthedocs.io/en/latest/javascript_api.html

## Advanced: Make an animation
With reactive options, you can easily implement animations in an Escher map. Here's a simple example where flux predictions are swept across and range to update the map of E. coli core metabolism.

In [ ]:
builder = Builder(
    height=600, 
    map_name=None, 
    model_name='e_coli_core',
    map_json='https://escher.github.io/1-0-0/5/maps/Escherichia%20coli/e_coli_core.Core%20metabolism.json',
)
builder

In [ ]:
model = cobra.io.load_json_model('e_coli_core.json')

In [ ]:
reverse = False
step = 0.1
timestep = 0.1
duration = 15 # seconds
lim = [0, 0.5]
val = lim[-1]
for _ in range(int(duration / timestep)):
    model.reactions.EX_o2_e.lower_bound = -val
    solution = model.optimize()
    builder.reaction_data = solution.fluxes
    if val <= lim[0]:
        reverse = True
    if val >= lim[1]:
        reverse = False
    if reverse:
        val += step
    else:
        val -= step
    sleep(timestep)

## Saving maps as HTML
The Escher Builder can also be saved as a standalone HTML file, which you can view by opening in a browser. Just provide a filepath, and the map will be bundled along with all the current options.

In [ ]:
builder.save_html('example_map.html')